In [ ]:
# TODO use actual imports here
from dummy import load_breast_cancer, Chain, Scaler, Repartitioner, \
    XGBoostTrainer, MLflowLoggerCallback, JsonLoggerCallback, TBXLoggerCallback, \
    StopperCallback

import optim

# tag::ds_create[]
from ray.data import from_pandas
import sklearn.datasets


data_raw = sklearn.datasets.load_breast_cancer(as_frame=True)  # <1>

dataset_df = data_raw["data"]
predict_ds = from_pandas(dataset_df)  # <2>

dataset_df["target"] = data_raw["target"]
dataset = from_pandas(dataset_df)
# end::ds_create[]

In [ ]:
# tag::preprocessor_create[]
preprocessor = Chain(  # <1>
    Scaler(["worst radius", "worst area"]),  # <2>
    Repartitioner(num_partitions=2)  # <3>
)
# end::preprocessor_create[]

In [ ]:
# tag::train_fit_basic[]
trainer = XGBoostTrainer(
    scaling_config={  # <1>
        "num_actors": 2,
        "gpus_per_actor": 0,
        "cpus_per_actor": 2,
    },
    label="target",  # <2>
    params={  # <3>
        "tree_method": "approx",
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
)

result = trainer.fit(dataset=dataset, preprocessor=preprocessor)  # <4>

print(result)
# end::train_fit_basic[]

In [ ]:
# tag::training_func_setup[]
import torch
import torch.nn as nn


num_samples = 20
input_size = 10
layer_size = 15
output_size = 5


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(layer_size, output_size)

    def forward(self, input_data):
        return self.layer2(self.relu(self.layer1(input_data)))


input = torch.randn(num_samples, input_size)  # <1>
labels = torch.randn(num_samples, output_size)


def train_func():  # <2>
    num_epochs = 3
    model = NeuralNetwork()
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    for epoch in range(num_epochs):
        output = model(input)
        loss = loss_fn(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
# end::training_func_setup[]

In [ ]:
# tag::dist_func_setup[]
import ray.train.torch


def train_func_distributed():
    num_epochs = 3
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)  # <1>
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    for epoch in range(num_epochs):
        output = model(input)
        loss = loss_fn(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
# end::dist_func_setup[]

In [ ]:
# tag::train_start[]
from ray.train import Trainer


trainer = Trainer(backend="torch", num_workers=4, use_gpu=False)  # <1>
trainer.start()
results = trainer.run(train_func_distributed)
trainer.shutdown()
# end::train_start[]

In [ ]:
# tag::xgb_start[]
scaling_config = {"num_workers": 10, "use_gpu": True}

trainer = ray.train.integrations.XGBoostTrainer(
    scaling_config=scaling_config,
    # ...
)
# end::xgb_start[]

In [ ]:
# tag::get_datasets[]
from typing import Dict
import torch
import torch.nn as nn

import ray
import ray.train as train
from ray.train import Trainer


def get_datasets(a=5, b=10, size=1000, split=0.8):

    def get_dataset(a, b, size):
        items = [i / size for i in range(size)]
        dataset = ray.data.from_items([{"x": x, "y": a * x + b} for x in items])
        return dataset

    dataset = get_dataset(a, b, size)
    split_index = int(dataset.count() * split)
    train_dataset, validation_dataset = dataset.random_shuffle().split_at_indices(
        [split_index]
    )
    train_dataset_pipeline = train_dataset.repeat().random_shuffle_each_window()
    validation_dataset_pipeline = validation_dataset.repeat()
    datasets = {
        "train": train_dataset_pipeline,
        "validation": validation_dataset_pipeline,
    }
    return datasets
# end::get_datasets[]

In [ ]:
# tag::train_func_with_data[]
def train_func(config):
    batch_size = config["batch_size"]
    # hidden_size = config["hidden_size"]
    # lr = config.get("lr", 1e-2)
    epochs = config.get("epochs", 3)

    train_dataset_pipeline_shard = train.get_dataset_shard("train")
    validation_dataset_pipeline_shard = train.get_dataset_shard("validation")
    train_dataset_iterator = train_dataset_pipeline_shard.iter_epochs()
    validation_dataset_iterator = validation_dataset_pipeline_shard.iter_epochs()

    for _ in range(epochs):
        train_dataset = next(train_dataset_iterator)
        validation_dataset = next(validation_dataset_iterator)
        train_torch_dataset = train_dataset.to_torch(
            label_column="y",
            feature_columns=["x"],
            label_column_dtype=torch.float,
            feature_column_dtypes=torch.float,
            batch_size=batch_size,
        )
        validation_torch_dataset = validation_dataset.to_torch(
            label_column="y",
            feature_columns=["x"],
            label_column_dtype=torch.float,
            feature_column_dtypes=torch.float,
            batch_size=batch_size,
        )
        # ... training

    return results
# end::train_func_with_data[]

In [ ]:
# tag::put_things_together[]
num_workers = 4
use_gpu = False


datasets = get_datasets()
trainer = Trainer("torch", num_workers=num_workers, use_gpu=use_gpu)
config = {"lr": 1e-2, "hidden_size": 1, "batch_size": 4, "epochs": 3}
trainer.start()
results = trainer.run(
   train_func,
   config,
   dataset=datasets,
   callbacks=[JsonLoggerCallback(), TBXLoggerCallback()],
)
trainer.shutdown()
print(results)

# end::put_things_together[]

In [ ]:
# tag::ray_tune_integration[]
from ray import tune

fail_after_finished = True
prep_v1 = preprocessor
prep_v2 = preprocessor

param_space = {
    "scaling_config": {
        "num_actors": tune.grid_search([2, 4]),
        "cpus_per_actor": 2,
        "gpus_per_actor": 0,
    },
    "preprocessor": tune.grid_search([prep_v1, prep_v2]),
    # "datasets": {
    #     "train_dataset": tune.grid_search([dataset_v1, dataset_v2]),
    # },
    "params": {
        "objective": "binary:logistic",
        "tree_method": "approx",
        "eval_metric": ["logloss", "error"],
        "eta": tune.loguniform(1e-4, 1e-1),
        "subsample": tune.uniform(0.5, 1.0),
        "max_depth": tune.randint(1, 9),
    },
}
if fail_after_finished > 0:
    callbacks = [StopperCallback(fail_after_finished=fail_after_finished)]
else:
    callbacks = None
tuner = tune.Tuner(
    XGBoostTrainer(
        run_config={"max_actor_restarts": 1},
        scaling_config=None,
        resume_from_checkpoint=None,
        label="target",
    ),
    run_config={},
    param_space=param_space,
    name="tuner_resume",
    callbacks=callbacks,
)
results = tuner.fit(datasets={"train_dataset": dataset})
print(results.results)
# end::ray_tune_integration[]

In [ ]:
# tag::trainer_callbacks[]


# Run the training function, logging all the intermediate results
# to MLflow and Tensorboard.

result = trainer.run(
    train_func,
    callbacks=[
        MLflowLoggerCallback(experiment_name="train_experiment"),
        TBXLoggerCallback(),
    ],
)
# end::trainer_callbacks[]

In [ ]:
# tag::export_model[]
result = trainer.fit(dataset=dataset, preprocessor=preprocessor)
print(result)

this_checkpoint = result.checkpoint
this_model = this_checkpoint.load_model()
predicted = this_model.predict(predict_ds)
print(predicted.to_pandas())
# end::export_model[]